[Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence)

#### Checkpoints

In [15]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

{'foo': 'b', 'bar': ['a', 'b']}

##### Get state

In [16]:
# get the latest state snapshot
config = {"configurable": {"thread_id": "1"}}
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8414-6970-8002-fa22be376fdc'}}, metadata={'source': 'loop', 'writes': {'node_b': {'foo': 'b', 'bar': ['b']}}, 'thread_id': '1', 'step': 2, 'parents': {}}, created_at='2025-03-25T12:35:38.715981+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8412-6bc0-8001-9b6e207d4213'}}, tasks=())

In [18]:
# get a state snapshot for a specific checkpoint_id
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f00975a-8414-6970-8002-fa22be376fdc"}}
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f00975a-8414-6970-8002-fa22be376fdc'}}, metadata={'source': 'loop', 'writes': {'node_b': {'foo': 'b', 'bar': ['b']}}, 'thread_id': '1', 'step': 2, 'parents': {}}, created_at='2025-03-25T12:35:38.715981+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8412-6bc0-8001-9b6e207d4213'}}, tasks=())

##### Get state history

In [19]:
config = {"configurable": {"thread_id": "1"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8414-6970-8002-fa22be376fdc'}}, metadata={'source': 'loop', 'writes': {'node_b': {'foo': 'b', 'bar': ['b']}}, 'thread_id': '1', 'step': 2, 'parents': {}}, created_at='2025-03-25T12:35:38.715981+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8412-6bc0-8001-9b6e207d4213'}}, tasks=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8412-6bc0-8001-9b6e207d4213'}}, metadata={'source': 'loop', 'writes': {'node_a': {'foo': 'a', 'bar': ['a']}}, 'thread_id': '1', 'step': 1, 'parents': {}}, created_at='2025-03-25T12:35:38.715224+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00975a-8410-6546-8000-7416a826ee87'}}, tasks=(Preg

![state_history](https://langchain-ai.github.io/langgraph/concepts/img/persistence/get_state.jpg)

##### Replay

In [20]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f00975a-8412-6bc0-8001-9b6e207d4213"}}
graph.invoke(None, config=config)

{'foo': 'b', 'bar': ['a', 'b']}

##### Update state

In [23]:
config = {"configurable": {"thread_id": "1"}}
graph.update_state(config, {"foo": 2, "bar": ["b"]})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f009775-2210-693e-8003-47f097915e83'}}

In [24]:
graph.get_state(config)

StateSnapshot(values={'foo': 2, 'bar': ['a', 'b', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f009775-2210-693e-8003-47f097915e83'}}, metadata={'source': 'update', 'writes': {'node_b': {'foo': 2, 'bar': ['b']}}, 'thread_id': '1', 'checkpoint_id': '1f00975a-8412-6bc0-8001-9b6e207d4213', 'step': 3, 'parents': {}}, created_at='2025-03-25T12:47:33.213989+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00976b-e8a6-6198-8002-ca97e8610fc9'}}, tasks=())

#### Memory Store

##### Basic Usage

In [2]:
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()

In [3]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

In [4]:
import uuid

memory_id = str(uuid.uuid4())
memory = {"food_preference" : "I like pizza"}
in_memory_store.put(namespace_for_memory, memory_id, memory)

In [6]:
memories = in_memory_store.search(namespace_for_memory)
memories[-1].dict()

{'namespace': ['1', 'memories'],
 'key': '59dbda24-a12a-412e-854a-cb39e94e5746',
 'value': {'food_preference': 'I like pizza'},
 'created_at': '2025-03-25T03:01:31.390967+00:00',
 'updated_at': '2025-03-25T03:01:31.390970+00:00',
 'score': None}

##### Semantic Search

基于embeddings的记忆存储和语义搜索

In [7]:
from langchain.embeddings import init_embeddings

store = InMemoryStore(
    index={
        "embed": init_embeddings("openai:text-embedding-3-small"),  # Embedding provider
        "dims": 1536,                              # Embedding dimensions
        "fields": ["food_preference", "$"]              # Fields to embed
    }
)

/var/folders/ls/m3p1b7qs2r329nz2vmtqp6cc0000gn/T/ipykernel_3584/3894971003.py:5: LangChainBetaWarning: The function `init_embeddings` is in beta. It is actively being worked on, so the API may change.
  "embed": init_embeddings("openai:text-embedding-3-small"),  # Embedding provider


In [14]:
# Find memories about food preferences
# (This can be done after putting memories into the store)
memories = store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)
memories


[Item(namespace=['1', 'memories'], key='e03281f9-9757-4891-b18d-617f12c31e6c', value={'food_preference': 'I love Italian cuisine', 'context': 'Discussing dinner plans'}, created_at='2025-03-25T12:25:33.410568+00:00', updated_at='2025-03-25T12:25:33.410577+00:00', score=0.37711601741226003),
 Item(namespace=['1', 'memories'], key='35dfaffb-4e27-44d0-985e-cd80937186e5', value={'system_info': 'Last updated: 2024-01-01'}, created_at='2025-03-25T12:25:33.410845+00:00', updated_at='2025-03-25T12:25:33.410846+00:00', score=None)]

In [9]:
# Store with specific fields to embed
store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {
        "food_preference": "I love Italian cuisine",
        "context": "Discussing dinner plans"
    },
    index=["food_preference"]  # Only embed "food_preferences" field
)

# Store without embedding (still retrievable, but not searchable)
store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {"system_info": "Last updated: 2024-01-01"},
    index=False
)

##### Using in LangGraph

the checkpointer saves state to threads, as discussed above, and the in_memory_store allows us to store arbitrary information for access across threads.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# We need this because we want to enable threads (conversations)
checkpointer = MemorySaver()

# ... Define the graph ...

# Compile the graph with the checkpointer and store
graph = graph.compile(checkpointer=checkpointer, store=in_memory_store)

In [ ]:
user_id = "1"
# We invoke the graph with a thread_id, as before, and also with a user_id, 
# which we'll use to namespace our memories to this particular user as we showed above.
config = {"configurable": {"thread_id": "1", "user_id": user_id}}

# First let's just say hi to the AI
for update in graph.stream(
    {"messages": [{"role": "user", "content": "hi"}]}, config, stream_mode="updates"
):
    print(update)

We can access the in_memory_store and the user_id in any node by passing store: BaseStore and config: RunnableConfig as node arguments. Here's how we might use semantic search in a node to find relevant memories:

In [ ]:
def update_memory(state: MessagesState, config: RunnableConfig, *, store: BaseStore):

    # Get the user id from the config
    user_id = config["configurable"]["user_id"]

    # Namespace the memory
    namespace = (user_id, "memories")

    # ... Analyze conversation and create a new memory

    # Create a new memory ID
    memory_id = str(uuid.uuid4())

    # We create a new memory
    store.put(namespace, memory_id, {"memory": memory})

As we showed above, we can also access the store in any node and use the store.search method to get memories. Recall the the memories are returned as a list of objects that can be converted to a dictionary.

In [ ]:
memories[-1].dict()
{'value': {'food_preference': 'I like pizza'},
 'key': '07e0caf4-1631-47b7-b15f-65515d4c1843',
 'namespace': ['1', 'memories'],
 'created_at': '2024-10-02T17:22:31.590602+00:00',
 'updated_at': '2024-10-02T17:22:31.590605+00:00'}

We can access the memories and use them in our model call.

In [ ]:
def call_model(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    # Get the user id from the config
    user_id = config["configurable"]["user_id"]

    # Namespace the memory
    namespace = (user_id, "memories")

    # Search based on the most recent message
    memories = store.search(
        namespace,
        query=state["messages"][-1].content,
        limit=3
    )
    info = "\n".join([d.value["memory"] for d in memories])

    # ... Use memories in the model call